In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [78.0 kB]
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,351 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubunt

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [ ]:
# Add the file to Spark using SparkFiles
spark.sparkContext.addFile(url)

In [ ]:
# Read the CSV file into a DataFrame
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [ ]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView("my_table")
print(home_sales_df)

DataFrame[id: string, date: timestamp, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]


In [ ]:
# Perform operations on the temporary view just to test
spark.sql("SELECT * FROM my_table").show()

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("SELECT date, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 4 GROUP BY date ORDER BY date")
result.show()

+-------------------+-------------+
|               date|average_price|
+-------------------+-------------+
|2019-01-01 00:00:00|     365088.0|
|2019-01-02 00:00:00|    271644.29|
|2019-01-03 00:00:00|    381218.86|
|2019-01-04 00:00:00|    359586.14|
|2019-01-05 00:00:00|    323683.57|
|2019-01-06 00:00:00|    301091.13|
|2019-01-07 00:00:00|     327474.5|
|2019-01-08 00:00:00|    304567.83|
|2019-01-09 00:00:00|     243619.0|
|2019-01-10 00:00:00|     287844.4|
|2019-01-11 00:00:00|    277574.29|
|2019-01-12 00:00:00|     218507.0|
|2019-01-13 00:00:00|    294603.86|
|2019-01-14 00:00:00|    284434.57|
|2019-01-15 00:00:00|     254288.5|
|2019-01-16 00:00:00|     335106.5|
|2019-01-17 00:00:00|    333439.09|
|2019-01-18 00:00:00|    297542.56|
|2019-01-19 00:00:00|    294484.22|
|2019-01-20 00:00:00|    312395.33|
+-------------------+-------------+
only showing top 20 rows



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
result.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
result.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
result = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM my_table WHERE price >= 350000")
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.9643919467926025 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("my_table")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('my_table')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 4.696846008300781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

# Specify the output path for the partitioned Parquet data
output_path = "path/to/partitioned_data.parquet"

# Write the DataFrame to Parquet format with partitioning
home_sales_df.write.partitionBy("date_built").parquet(output_path)

In [ ]:
# 11. Read the parquet formatted data.
# Specify the path to the formatted Parquet data
parquet_path = "path/to/partitioned_data.parquet"

# Read the Parquet data into a DataFrame
parquet_data = spark.read.parquet(parquet_path)
parquet_data

DataFrame[id: string, date: timestamp, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_table")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
# Run the query on the cached DataFrame
result_cached = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM my_table WHERE view >= 350000")
result_cached.show()

cached_runtime = time.time() - start_time


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|         null|
+-------------+

--- 0.9125568866729736 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("my_table")

In [ ]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("my_table")

if is_cached:
    print("The 'my_table'  is still cached.")
else:
    print("The 'my_table'  is no longer cached.")

The 'my_table'  is no longer cached.
